In [27]:
# compute document similarity on the test data. Use average word embeddings.
import os, pprint, json, pickle
import gensim, spacy
import numpy as np
path_data = "/data/cm/data/"
path = "/home/rovera/cm/"

In [2]:
we = gensim.models.KeyedVectors.load_word2vec_format(path_data+'word_emb/cc.de.300.vec', binary=False)

In [5]:
nlp = spacy.load("de_core_news_lg")

In [30]:
doc_embeddings = {}
c = 0
for j in os.listdir(path_data+"pages/"):
    
    for file in os.listdir(path_data+"pages/"+j+"/"):
        
        if c % 100 == 0:
            print()
            print(c, end=" "*(5-len(str(c))))
        
        
        text = open(path_data+"pages/"+j+"/"+file, 'r', encoding="utf-8").read()
        doc = nlp(text)
        
        tokens = []
        
        for token in doc:
            
            if not token.is_stop:
                word = token.text
                if word in we:
                    tokens.append(we[word])
        if len(tokens) > 0:
            doc_embedding = sum(tokens)/len(tokens)
            doc_embeddings[file] = doc_embedding
        c += 1
        print("|", end="")


0    ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
100  ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
200  ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
300  ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
400  ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
500  ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
600  ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
700  ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
800  ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
900  ||||||||||||||||||||||||||||||||||||||||

In [17]:
print(list(doc_embeddings.keys())[:10])

['3133277_3133285__409_3133716.txt', '3133277_3133285__406_3133713.txt', '3133277_3133286__436_3133744.txt', '3133277_3133283__278_3133583.txt', '3133277_3133284__357_3133663.txt', '3133277_3133286__452_3133760.txt', '3133277_3133285__378_3133685.txt', '3133277_3133284__309_3133615.txt', '3133277_3133278__050_3133350.txt', '3133277_3133278__003_3133303.txt']


In [31]:
from sklearn.metrics.pairwise import cosine_similarity as cos

In [33]:
doc_embeddings = {k: np.reshape(v, (1, -1)) for k,v in doc_embeddings.items()}

In [35]:
# compute similarity for a single page with all other pages
# 3133277_3133284__309_3133615.txt
similarity = {}
query = doc_embeddings['3133277_3133284__309_3133615.txt']
for token, vec in doc_embeddings.items():
    sim = cos(query, vec)
    similarity[token] = sim[0][0]

In [39]:
sorted_sim = sorted([(k,v) for k,v in similarity.items()], key=lambda x:x[1], reverse=True)

In [42]:
pprint.pprint(sorted_sim[:20])

[('3133277_3133284__309_3133615.txt', 0.9999999),
 ('3133277_3133284__345_3133651.txt', 0.97836494),
 ('3133277_3133278__045_3133345.txt', 0.9779254),
 ('2566635_2566639__44_2566703.txt', 0.9778295),
 ('2566635_2566636__11_2566667.txt', 0.97734284),
 ('3133277_3133280__140_3133442.txt', 0.9771153),
 ('2566635_2566636__12_2566668.txt', 0.9770877),
 ('3133277_3133285__399_3133706.txt', 0.97665846),
 ('3133277_3133281__162_3133465.txt', 0.9763735),
 ('3133277_3133278__015_3133315.txt', 0.9756376),
 ('9633717_9625934__460_9625938.txt', 0.9755571),
 ('9632912_9625822__346_9625825.txt', 0.9754943),
 ('2566635_2566639__43_2566702.txt', 0.97539693),
 ('2566635_2566643__78_2566741.txt', 0.9753741),
 ('3133277_3133286__435_3133743.txt', 0.975283),
 ('3133277_3133283__252_3133557.txt', 0.9752318),
 ('3133277_3133285__381_3133688.txt', 0.97504145),
 ('3133277_3133284__324_3133630.txt', 0.97502524),
 ('3133277_3133280__127_3133429.txt', 0.97472715),
 ('3133277_3133280__138_3133440.txt', 0.9746584)]